<center>

<h1 style="text-align:center"> Lambda Calculus : Encodings </h1>
<h2 style="text-align:center"> CS3100 Fall 2019 </h2>
</center>

## Power of Lambdas

* Despite its simplicity, the lambda calculus is quite expressive: it is **Turing complete**!
* Means we can encode any computation we want
  + if we are sufficiently clever...
* Examples
  + Booleans & predicate logic.
  + Pairs
  + Lists
  + Natural numbers & arithmetic.
  
$\newcommand{\br}{\rightarrow_{\beta}}$

In [ ]:
#use "init.ml"

let p = Lambda_parse.parse_string                                                   
let var x = Var x                                                                 
let app l =
  match l with 
  | [] -> failwith "ill typed app"
  | [x] -> x
  | x::y::xs -> List.fold_left (fun expr v -> App (expr, v)) (App(x,y)) xs
let lam x e = Lam(x,e)                                                              
                                                                                    
let eval ?(log=true) s = 
     s  
  |> Eval.eval ~log Eval.reduce_normal 
  |> Syntax.string_of_expr

## Booleans

In [ ]:
let tru = p "\\t.\\f.t"
let fls = p "\\t.\\f.f"

* Now we can define a `test` function such that
  + `test tru v w` $\br$ `v`
  + `test fls v w` $\br$ `w`

In [ ]:
let test = p "\\l.\\m.\\n.l m n"

## Booleans

Now 

```ocaml
test tru v w
```

evaluates to

In [ ]:
eval @@ app [test; tru; var "v"; var "w"]

## Booleans

Similarly,

```ocaml
test fls v w
```

evaluates to

In [ ]:
eval @@ app [test; fls; var "v"; var "w"]

## Booleans

`fls` itself is a function. `test fls v w` is equivalent to `fls v w`.

In [ ]:
eval @@ app [fls; var "v"; var "w"]

## Logical operators

```ocaml
and = λb.λc.b c fls
or = λb.λc.b tru c
not = λb.b fls tru
```

In [ ]:
let and_ = lam "b" (lam "c" (app [var "b"; var "c"; fls]))
let or_ = lam "b" (lam "c" (app [var "b"; tru; var "c"]))
let not_ = lam "b" (app [var "b"; fls; tru])

## Logical Operators

In [ ]:
eval @@ app [and_; tru; fls]

The above is a **proof** for `true /\ false = false`

## Logical operators

\\[
\begin{array}{rl}
 & p \implies q \equiv \neg p \vee q \\
\mathbf{Theorem 1.}  & a \wedge b \implies a
\end{array}
\\]

In [ ]:
let implies = lam "p" (lam "q" (app [or_; app [not_; var "p"]; var "q"]))
let thm1 = lam "a" (lam "b" (app [implies; app [and_; var "a"; var "b"]; var "a"]))

## Logical operators

In [ ]:
eval ~log:false (app [thm1; var "x"; var "y"])

## Quiz

What is the lambda calculus encoding for `xor x y`

| x | y | xor x y |
|---|---|:-------:|
| T | T |    F    |
| T | F |    T    |
| F | T |    T    |
| F | F |    F    |

1. x x y
2. x (y tru fls) y
3. x (y fls tru) y
4. y x y

## Quiz

What is the lambda calculus encoding for `xor x y`

| x | y | xor x y |
|---|---|:-------:|
| T | T |    F    |
| T | F |    T    |
| F | T |    T    |
| F | F |    F    |

1. x x y
2. x (y tru fls) y
3. x (y fls tru) y ✅
4. y x y

## Pairs

* Encoding of a pair `(a,b)`
  + (a,b) = λf.λs.λb.b f s
  + fst = λf.f tru
  + snd = λf.f fls

In [22]:
let pair = p ("λf.λs.λb.b f s")
let fst = lam "p" (app [var "p"; tru])
let snd = lam "p" (app [var "p"; fls])

val pair : Syntax.expr =
  Lam ("f", Lam ("s", Lam ("b", App (App (Var "b", Var "f"), Var "s"))))


val fst : Syntax.expr =
  Lam ("p", App (Var "p", Lam ("t", Lam ("f", Var "t"))))


val snd : Syntax.expr =
  Lam ("p", App (Var "p", Lam ("t", Lam ("f", Var "f"))))


## Pairs

In [23]:
eval @@ app [fst; app [pair; var "v"; var "w"]]

= (λf.λs.λb.b f s) v w (λt.λf.t)
= (λs.λb.b v s) w (λt.λf.t)
= (λb.b v w) (λt.λf.t)
= (λt.λf.t) v w
= (λf.v) w
= v


- : string = "v"


## Natural numbers

* 0 = λs.λz.z
* 1 = λs.λz.s z
* 2 = λs.λz.s (s z)
* 3 = λs.λz.s (s (s z))

i.e., n = λs.λz.(apply `s` n times to `z`)

Also known as **Church numerals**.

## Natural numbers

In [25]:
let zero = p ("λs.λz.z")
let one = p ("λs.λz.s z")
let two = p ("λs.λz.s (s z)")
let three = p ("λs.λz.s (s (s z))")

val zero : Syntax.expr = Lam ("s", Lam ("z", Var "z"))


val one : Syntax.expr = Lam ("s", Lam ("z", App (Var "s", Var "z")))


val two : Syntax.expr =
  Lam ("s", Lam ("z", App (Var "s", App (Var "s", Var "z"))))


val three : Syntax.expr =
  Lam ("s", Lam ("z", App (Var "s", App (Var "s", App (Var "s", Var "z")))))


## Quiz

What will be the OCaml type of church encoded numeral?

1. `('a -> 'b) -> 'a -> 'b`
2. `('a -> 'a) -> 'a -> 'a` 
3. `('a -> 'a) -> 'b -> int`
4. `(int -> int) -> int -> int`

## Quiz

What will be the OCaml type of church encoded numeral?

1. `('a -> 'b) -> 'a -> 'b`
2. `('a -> 'a) -> 'a -> 'a` ✅
3. `('a -> 'a) -> 'b -> int`
4. `(int -> int) -> int -> int`

## Operations on numbers: Successor

Successor function is:

```ocaml
scc = λn.λs.λz.s (n s z)
```

In [27]:
let scc = p ("λn.λs.λz.s (n s z)")

val scc : Syntax.expr =
  Lam ("n",
   Lam ("s", Lam ("z", App (Var "s", App (App (Var "n", Var "s"), Var "z")))))


In [31]:
eval @@ app [scc; zero]

= λs.λz.s ((λs.λz.z) s z)
= λs.λz.s ((λz.z) z)
= λs.λz.s z


- : string = "λs.λz.s z"


## Operations on numbers : is_zero

Check if the given number is zero:

```ocaml
is_zero = λn.n (λy.fls) tru
```

In [32]:
let is_zero = lam "n" (app [var "n"; lam "y" fls; tru])

val is_zero : Syntax.expr =
  Lam ("n",
   App (App (Var "n", Lam ("y", Lam ("t", Lam ("f", Var "f")))),
    Lam ("t", Lam ("f", Var "t"))))


## Operations on numbers : is_zero

In [35]:
eval @@ app [is_zero; zero]

= (λs.λz.z) (λy.λt.λf.f) (λt.λf.t)
= (λz.z) (λt.λf.t)


- : string = "λt.λf.t"


= λt.λf.t


In [36]:
eval @@ app [is_zero; one]

= (λs.λz.s z) (λy.λt.λf.f) (λt.λf.t)


- : string = "λt.λf.f"


= (λz.(λy.λt.λf.f) z) (λt.λf.t)
= (λy.λt.λf.f) (λt.λf.t)
= λt.λf.f


## Arithmetic

```ocaml
plus = λm.λn.λs.λz.m s (n s z)
mult = λm.λn.λs.λz.m n s z
```

In [40]:
let plus = p ("λm.λn.λs.λz.m s (n s z)")
let mult = p ("λm.λn.λs.λz.m n s z")

val plus : Syntax.expr =
  Lam ("m",
   Lam ("n",
    Lam ("s",
     Lam ("z",
      App (App (Var "m", Var "s"), App (App (Var "n", Var "s"), Var "z"))))))


val mult : Syntax.expr =
  Lam ("m",
   Lam ("n",
    Lam ("s",
     Lam ("z", App (App (App (Var "m", Var "n"), Var "s"), Var "z")))))


## Arithmetic: addition

In [41]:
eval @@ app [plus; one; two]

= (λn.λs.λz.(λs.λz.s z) s (n s z)) (λs.λz.s (s z))
= λs.λz.(λs.λz.s z) s ((λs.λz.s (s z)) s z)
= λs.λz.(λz.s z) ((λs.λz.s (s z)) s z)
= λs.λz.s ((λs.λz.s (s z)) s z)
= λs.λz.s ((λz.s (s z)) z)
= λs.λz.s (s (s z))


- : string = "λs.λz.s (s (s z))"


Proves 1 + 2 = 3. Can build a theory of arithmetic over lambda calculus.

## Arithmetic: multiplication

In [44]:
eval @@ app [mult; one; two]

- : string = "λs.λz.s (s z)"


= (λn.λs.λz.(λs.λz.s z) n s z) (λs.λz.s (s z))
= λs.λz.(λs.λz.s z) (λs.λz.s (s z)) s z
= λs.λz.(λz.(λs.λz.s (s z)) z) s z
= λs.λz.(λs0.λz.s0 (s0 z)) s z
= λs.λz.(λz.s (s z)) z
= λs.λz.s (s z)


## Todo

* pred, sub, equal
* looping, fixpoints and recursion
* lists and trees
* Need for typing